In [29]:
#Import libraries
import json
import requests
from IPython.display import JSON
import pandas as pd 

#Download the astronaut database from SuperCluster
astronaut_db_url = 'https://supercluster-iadb.s3.us-east-2.amazonaws.com/adb.json'
astronauts_db = requests.get(astronaut_db_url).json()

#Make dataframes
df1 = pd.json_normalize(astronauts_db['astronauts'])
df2 = pd.json_normalize(astronauts_db['missions'])

#Grab columns
df_astro = df1[['_id','astroNumber','awards','name','gender','inSpace','overallNumber','spacewalkCount','species','speciesGroup',
                'totalMinutesInSpace','totalSecondsSpacewalking','lastLaunchDate.utc']]

df_miss = df2[['_id','astronauts','keywords','name',
               'seriesName','shortDescription','vagueLaunchDate',
               'landDate.utc','launchDate.utc']]

#Check shapes of dfs
print(df_astro.shape)
print(df_miss.shape)

#Change column names
df_astro = df_astro.rename(columns={'_id': 'astronaut_id'})

#Get row per award
df_awards = df_astro[['astronaut_id', 'awards']].copy()
df_awards['awards'] = df_awards['awards'].apply(lambda awards: [award['title'] for award in awards])
df_awards.head(2)

#Join awards column back on astronaut df
df_astro = pd.merge(df_astro,df_awards,how='left',on=['astronaut_id'])

#Clean up astronaut df
del df_astro['awards_x']
df_astro = df_astro.rename(columns={'awards_y': 'awards'})


#Change column names
df_miss = df_miss.rename(columns={'_id': 'mission_id'})

#Expand df to have multiple rows (many astronauts per mission)
df_test = df_miss.explode(['astronauts']).reset_index(drop=True)


#Pull out list of astronauts from JSON format
astronauts = pd.json_normalize(df_test['astronauts'])


#Add list of astronauts back into mission df
df_miss = pd.concat([df_test, astronauts], axis=1)

#Change column names
df_miss = df_miss.rename(columns={'_id': 'astronaut_id'})
del df_miss['astronauts']

#Cleaning time/day variables
df_miss['launch_time'] = pd.to_datetime(df_miss['launchDate.utc']).dt.time
df_miss['land_time'] = pd.to_datetime(df_miss['landDate.utc']).dt.time
df_miss['launch_date'] = df_miss['vagueLaunchDate']
df_miss['land_date'] = pd.to_datetime(df_miss['landDate.utc']).dt.date

del df_miss['vagueLaunchDate']
del df_miss['landDate.utc']
del df_miss['launchDate.utc']

#Join astronaut database with mission database
df_full = pd.merge(df_miss,df_astro,how='left',on=['astronaut_id'])

print(df_miss.shape)
print(df_astro.shape)
print(df_full.shape)

(952, 13)
(490, 9)
(1498, 10)
(952, 13)
(1498, 22)


In [25]:
#full_df.to_csv(r'/Users/jonzimmerman/Desktop/Data Projects/Astronaut Database/astro_db.csv', index = False)


In [31]:
df_full.head(1)

,mission_id,keywords,name_x,seriesName,shortDescription,astronaut_id,launch_time,land_time,launch_date,land_date,...,gender,inSpace,overallNumber,spacewalkCount,species,speciesGroup,totalMinutesInSpace,totalSecondsSpacewalking,lastLaunchDate.utc,awards
0,00d2e869-4c3d-4be3-b2e9-c3a935768abb,"SpaceX, Crewed",SpaceX Crew-2,SpaceX,Crew-2 is SpaceX's second operational flight o...,NSZm43AoLYq0PEOXpgALIz,09:49:00,03:33:00,2021-04-23,2021-11-09,...,male,False,746.0,9.0,Human,human,559768.0,213960.0,2021-04-23T00:00:00.000Z,"[ISS Visitor, Crossed Kármán Line, Elite Space..."


In [42]:
len(df_full['awards'][0])

6

In [44]:
SpaceX = df_full[df_full['seriesName']=="SpaceX"]
SpaceX.shape

(18, 22)

In [49]:
df_full['speciesGroup'].unique()

array(['human', nan], dtype=object)